# Libraries and Referenced Functions

In [1]:
#https://www.kaggle.com/c/kobe-bryant-shot-selection

In [1]:
# https://www.kaggle.com/dixhom/data-analysis-for-beginners

import pandas as pd
import sklearn as sk
import numpy as np
import matplotlib.pyplot as plt
import time
from matplotlib.patches import Circle, Rectangle, Arc
import matplotlib.cm as cm
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.tree import DecisionTreeClassifier
import random
import seaborn as sns
%matplotlib inline

random.seed(123)

In [2]:
df_raw = pd.read_csv("data.csv")
print(df_raw.shape)
df = df_raw.copy()

df = df[pd.notnull(df['shot_made_flag'])]

df['secs_from_period_end']   = 60*df['minutes_remaining']+df['seconds_remaining']
df['secs_from_period_start'] = 60*(11-df['minutes_remaining'])+(60-df['seconds_remaining'])
df['secs_from_start']   = (df['period'] <= 4).astype(int)*(df['period']-1)*12*60 + (df['period'] > 4).astype(int)*((df['period']-4)*5*60 + 3*12*60) + df['secs_from_period_start']
df['dist'] = np.sqrt(df['loc_x']**2 + df['loc_y']**2)
df['away/home'] = df['matchup'].str.contains('vs').astype('int')
df['game_date'] = pd.to_datetime(df['game_date'])
df['game_year'] = df['game_date'].dt.year

loc_x_zero = (df['loc_x'] == 0)

df['angle'] = np.array([0]*len(df))
df['angle'][~loc_x_zero] = np.arctan(df['loc_y'][~loc_x_zero] / df['loc_x'][~loc_x_zero])
df['angle'][loc_x_zero] = np.pi / 2

to_drop = ['period', 'minutes_remaining', 'seconds_remaining', 'team_id', 'team_name', 'matchup', 'lon', 'lat',
           'game_id', 'game_event_id', 'game_date', 'playoffs', 'shot_distance', 'secs_from_period_start', 'season', 'game_date']

#df.set_index('shot_id', inplace=True)

df = df.drop(to_drop, axis = 1)

# under_10_secs = (df['secs_from_period_end'] < 10)==True
# under_10 = [int(i) for i in under_10_secs]
# last_10_bool = df[under_10_secs]
# df['last_10s'] = under_10

categorical_cols = ['action_type', 'combined_shot_type', 'shot_type', 'shot_zone_area',
                        'shot_zone_basic', 'shot_zone_range', 'opponent']

for col in categorical_cols:
    dummies = pd.get_dummies(df[col])
    dummies = dummies.add_prefix("{}_".format(col))
    df.drop(col, axis=1, inplace=True)
    df = df.join(dummies)
    

(30697, 25)


C:\Users\bubba_000\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\bubba_000\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [21]:
print(df_raw.shape, df_raw.columns)
print()
df['game_year'].unique()
#df_raw[['game_date', 'season']]

(30697, 25) Index(['action_type', 'combined_shot_type', 'game_event_id', 'game_id', 'lat',
       'loc_x', 'loc_y', 'lon', 'minutes_remaining', 'period', 'playoffs',
       'season', 'seconds_remaining', 'shot_distance', 'shot_made_flag',
       'shot_type', 'shot_zone_area', 'shot_zone_basic', 'shot_zone_range',
       'team_id', 'team_name', 'game_date', 'matchup', 'opponent', 'shot_id'],
      dtype='object')



array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 1996, 1997, 1998, 1999],
      dtype=int64)

In [12]:
# cat = ['combined_shot_type', 'shot_type', 'shot_zone_area', 'shot_zone_basic', 'shot_zone_range', 'playoffs', 'opponent']
# cat_acc = []
# for i in cat:
#     cat_acc.append(df_raw.groupby(i)['shot_made_flag'].mean())

# for i in cat_acc:
#     print('\n\n',i)

In [13]:
# #c=df_raw.groupby('playoffs')['shot_made_flag'].mean()
# c=df_raw.groupby('combined_shot_type')['shot_made_flag'].mean()
# plt.figure()
# plt.bar(c.index, c.values)
# c

In [6]:
# c = df_raw.groupby('minutes_remaining')['shot_made_flag'].mean()
# plt.plot(c.index, c.values)
# plt.xlabel('Mins Remaining', size='xx-large')
# plt.ylabel('Shot accuracy', size='xx-large')
# plt.scatter(c.index, c.values)
# plt.savefig('accuracy_vs_mins_remaining')

In [7]:
# c = df_raw.groupby('shot_distance')['shot_made_flag'].mean()
# plt.plot(c.index, c.values, 'g')
# plt.xlabel('Shot_distance', size='xx-large')
# plt.ylabel('Shot accuracy', size='xx-large')
# plt.savefig('accuracy_vs_shot_distance')

In [17]:
# alpha = 0.02
# plt.figure(figsize=(10,5))

# # loc_x and loc_y
# plt.subplot(121)
# plt.scatter(df_raw.loc_x, df_raw.loc_y, color='green', alpha=0.01)
# plt.title('loc x and loc y', size = 'xx-large')

# # lat and lon
# plt.subplot(122)
# plt.scatter(df_raw.lon, df_raw.lat, color='red', alpha=0.01)
# plt.title('lat and lon', size = 'xx-large')
# plt.savefig('locxy_vs_latlon')


In [3]:
y = df['shot_made_flag'].copy()
X = df.drop(['shot_made_flag'], axis = 1)

x_tr, x_test, y_tr, y_test = train_test_split(np.array(X), np.array(y), test_size = 0.33, random_state=123)

### Cross Validation and Combining Algorithm Results

In [14]:
rfc=RandomForestClassifier(n_estimators=100, max_depth=10)
pred=rfc.fit(x_tr, y_tr).predict(x_test)
accuracy_score(y_test,pred)
print(0.6851+0.6878+0.6847)

print(0.6852+0.6836+0.6857)

2.0576
2.0545


In [17]:
models = []
preds = []
models.append(('Logistic Reg', LogisticRegression()))
models.append(('Random Forest', RandomForestClassifier(n_estimators=100, max_depth=10)))
models.append(('Bernoulli NB', BernoulliNB()))
models.append(('Gaussian NB', GaussianNB()))
models.append(('Gradient Boosting', GradientBoostingClassifier()))


for name, model in models:
    #CrossValidator()
    preds.append(model.fit(x_tr, y_tr).predict(x_test))

C:\Users\bubba_000\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [18]:
master_pred = sum(preds)

for i in range(len(master_pred)):
    if master_pred[i] >= len(preds)/2:
        master_pred[i] = 1
    else:
        master_pred[i] = 0


#preds.append(master_pred)

for i, pred in enumerate(preds):
    print(models[i][0], 'accuracy:', round(accuracy_score(y_test, pred), 3), '| F-score:', round(f1_score(y_test, pred), 3))
print('Combined accuracy:',round(accuracy_score(y_test, master_pred), 3), '| F-score:', round(f1_score(y_test, master_pred), 3))

Logistic Reg accuracy: 0.684 | F-score: 0.575
Random Forest accuracy: 0.686 | F-score: 0.577
Bernoulli NB accuracy: 0.656 | F-score: 0.559
Gaussian NB accuracy: 0.642 | F-score: 0.513
Gradient Boosting accuracy: 0.687 | F-score: 0.577
Combined accuracy: 0.684 | F-score: 0.578


In [12]:
# weighted combination
names = []

In [43]:
gbc = GradientBoostingClassifier()
m = gbc.fit(x_tr, y_tr).predict(x_test)


In [42]:
accuracy_score(m, y_test)

0.686947293951185

### Measuring Importance of Features

In [ ]:
threshold = 0.90
vt = VarianceThreshold().fit(x_tr)
rfc =RandomForestClassifier(n_estimators=100, max_depth=10).fit(x_tr, y_tr)

# Most important features
feat_var_threshold = X.columns[vt.variances_ > threshold * (1-threshold)]
feat_var_threshold
feature_imp = pd.DataFrame(rfc.feature_importances_, index=X.columns, columns=["importance"])

In [43]:
f=feature_imp.sort_values('importance', ascending=False)
f

,importance
action_type_Jump Shot,0.189189
dist,0.074334
action_type_Layup Shot,0.063115
combined_shot_type_Dunk,0.052239
combined_shot_type_Jump Shot,0.043400
loc_y,0.039048
secs_from_period_end,0.036629
shot_id,0.035782
action_type_Driving Layup Shot,0.031931
secs_from_start,0.031899
